In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, isnan
from pyspark.ml.feature import StringIndexer

# Step 1: Create Spark session
spark = SparkSession.builder \
    .appName("Data Cleaning") \
    .getOrCreate()

# Step 2: Load the dataset
file_path = r'C:\Users\thinu\Desktop\KENULA\top-up\data\Metro_Interstate_Traffic_Volume.csv'
data = spark.read.csv(file_path, header=True, inferSchema=True)


# Step 4: Data Cleaning

# 4.1: Identify and remove duplicate rows
# Step 3: Identify duplicate rows
duplicates = data.groupBy(data.columns).count().filter("count > 1")
print("Duplicate Rows:")
duplicates.show()

# Step 4: Remove duplicate rows
data = data.dropDuplicates()
print("Duplicate successfully removed")

# Order by date 
data = data.orderBy("date_time")
print("Date time oder successfully")

# Step 3: Identify duplicate rows
duplicates = data.groupBy(data.columns).count().filter("count > 1")
print("Duplicate Rows:")
duplicates.show()

# 4.2: Identify and handle null or missing values

# Separate numeric and non-numeric columns
numeric_columns = [field.name for field in data.schema.fields if str(field.dataType) in ['DoubleType', 'IntegerType', 'FloatType']]
non_numeric_columns = [field.name for field in data.schema.fields if field.name not in numeric_columns]

# Check nulls for all columns, and isnan only for numeric columns
null_counts = data.select([
    count(when(col(c).isNull(), c)).alias(c) for c in non_numeric_columns
] + [
    count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in numeric_columns
])
print("Null Counts Per Column:")
null_counts.show()

# Step 5: Export the cleaned and transformed dataset to CSV
output_path = r'C:\Users\thinu\Desktop\KENULA\top-up\data\cleaned_transformed_dataset2.csv'

try:
    data.toPandas().to_csv(output_path, index=False)
    print(f"Cleaned and transformed dataset saved successfully at {output_path}")
except Exception as e:
    print(f"Error saving dataset: {e}")

# Step 6: Stop the Spark session
spark.stop()

Duplicate Rows:
+-------+-------+-------+-------+----------+------------+--------------------+---------------+--------------+-----+
|holiday|   temp|rain_1h|snow_1h|clouds_all|weather_main| weather_description|      date_time|traffic_volume|count|
+-------+-------+-------+-------+----------+------------+--------------------+---------------+--------------+-----+
|   None| 292.84|    0.0|    0.0|         1|       Clear|        sky is clear|6/30/2017 10:00|          4638|    2|
|   None| 286.29|    0.0|    0.0|         1|       Clear|        sky is clear|9/30/2015 19:00|          3679|    2|
|   None| 296.95|    0.0|    0.0|         1|Thunderstorm|        thunderstorm|6/30/2017 13:00|          5263|    2|
|   None| 287.86|    0.0|    0.0|         0|       Clear|        Sky is Clear|9/29/2016 19:00|          3435|    2|
|   None| 288.89|    0.0|    0.0|         1|       Clear|        sky is clear|9/29/2017 19:00|          4498|    2|
|   None|289.775|    0.0|    0.0|        56|      Clouds